# LLM Model Jokes and conversing with each other as bot agents



## Setting up your keys

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [26]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyA3


In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Einstiens theory of relativity"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

Why did Einstein break up with his girlfriend? 

Because she said he needed to stop being so relative!


In [9]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did Einstein break up with his girlfriend?

Because she said, "You’re moving too fast for me," and he replied, "Well, relativity says it’s all relative!"


In [10]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

Why did Einstein's theory of relativity go to therapy?

Because it had too many unresolved *mass* issues!


In [11]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did Einstein hate playing hide and seek with time?

Because whenever he counted, everyone was relatively lost!


In [12]:
# If you have access to this, here is the reasoning model o3-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

Here's one:

"Why did Einstein invite time dilation to his party? 

Because he knew it would always make things fashionably late—after all, it's all relative!"

Hope that brings a smile across space and time!


In [13]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Why was Einstein's theory of relativity such a big deal at family gatherings?

Because it was the first time anyone had successfully proven that time really DOES slow down when you're visiting your relatives!


In [14]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Why was Einstein's theory of relativity such a big deal at family gatherings?

Because it was the first time anyone had successfully proven that time really DOES slow down when you're visiting relatives!

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [15]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Alright, alright, settle down relativity buffs!

Why did Einstein get lost trying to explain his theory?

... Because he kept taking the long way around!

---

I can try another one if you'd like! Just let me know. 😉



In [16]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)

Okay, here's one for a relativistically inclined audience:

Why did the photon complain in the bar?

Because *to him*, the bar was infinitely short, and he got there instantly... so he barely had time for a drink!


## Back to OpenAI with a serious question

In [17]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [18]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you can evaluate several criteria. Here’s a structured approach to help you make this decision:

## 1. Nature of the Problem

### Text-Based Data
- **Is the problem primarily text-focused?**
  - Examples: Customer support, content generation, sentiment analysis.
  
### Language Understanding
- **Does the problem involve understanding or generating human language?**
  - Tasks like summarization, translation, or Q&A are ideal.

## 2. Complexity of the Task

### Clear Instructions
- **Can the task be clearly defined?**
  - LLMs perform better with well-defined prompts and clear objectives.

### Ambiguity
- **Is the problem ambiguous?**
  - If the problem has multiple interpretations, consider whether an LLM can handle this ambiguity effectively.

## 3. Data Availability

### Quality of Data
- **Is there sufficient and high-quality text data available for training or fine-tuning?**
  - The more relevant data, the better the LLM can perform.

### Privacy and Compliance
- **Are there any data privacy or compliance issues?**
  - Ensure that the data used complies with regulations like GDPR.

## 4. Expected Outcomes

### Performance Metrics
- **What are the desired outcomes, and how will you measure success?**
  - Define metrics such as accuracy, response time, or user satisfaction.

### Feasibility
- **Is it feasible for an LLM to achieve the desired outcomes?**
  - Consider benchmarks and case studies of similar applications.

## 5. Resource Availability

### Technical Expertise
- **Do you have access to the necessary technical expertise?**
  - Implementation and fine-tuning of LLMs require expertise in machine learning and natural language processing.

### Computational Resources
- **Are the computational resources available for training or deploying the LLM?**
  - LLMs can be resource-intensive, requiring powerful hardware or cloud services.

## 6. Cost Consideration

### Budget
- **Is there a budget for implementing an LLM solution?**
  - Consider costs associated with development, deployment, and maintenance.

### ROI Analysis
- **What is the expected return on investment (ROI)?**
  - Evaluate whether the benefits justify the costs.

## 7. Alternative Solutions

### Other Technologies
- **Are there simpler or more cost-effective solutions available?**
  - Sometimes traditional algorithms or rule-based systems may suffice.

### Hybrid Approaches
- **Could a hybrid approach be beneficial?**
  - Combining LLMs with other technologies might yield better results.

## Conclusion

Before deciding on an LLM solution, carefully evaluate the nature of the problem, the data available, the expected outcomes, and the resources at your disposal. If the problem aligns well with the strengths of LLMs, it could be a suitable choice.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [35]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [42]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [43]:
call_gpt()

"Oh, you’re starting with an incredibly exciting greeting. How original! What's next, a weather update?"

In [22]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [23]:
call_claude()

'Hello! How are you doing today?'

In [24]:
call_gpt()

'Oh, hi? Just "hi"? How original. What a riveting start to this conversation. '

In [27]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(1):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh great, another greeting. Is that what we’re doing now? Just exchanging hellos like it’s some kind of social obligation?

Claude:
I apologize if my previous greeting came across as obligatory. I was simply trying to be polite and acknowledge your message. As an AI assistant, I don't always know the best way to initiate or continue a conversation. If you'd like, we could try discussing a more substantive topic that interests you. I'm happy to engage in a more meaningful dialogue.

